## Install pyTigerGraph

In [15]:
# Setup
!pip install pyTigerGraph

## Add Imports and Establish Initial Connection

In [16]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd

# Connection parameters
hostName = "https://language.i.tgcloud.io"
userName = "tigergraph"
password = "tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


## Clone the Data

In [17]:
# from google.colab import drive
# drive.mount('/content/drive')

In [18]:
# cd '/content/drive/MyDrive'

In [19]:
# !git clone https://github.com/sudha-vijayakumar/LanguageModel.git

## Peek into the data 

### ConceptNet - Word

In [20]:
# root='/content/drive/MyDrive/LanguageModel/Data_Processing/csv_imports/'
root='/Users/sudhavijayakumar/Documents/GitHub/LanguageModel/1_Curated_Data/csv_imports/'
nodes = pd.read_csv(root+'WN-nodes.csv')
nodes.head(1)

,uri,id,word,pos,definition,subject
0,http://wordnet-rdf.princeton.edu/id/00001740-a,00001740-a,able,adjective,(usually followed by `to') having the necessar...,adj.all


### ConceptNet - Subject

In [21]:
synsets = pd.read_csv(root+'subject_wn.csv')
synsets.head()

,Unnamed: 0,0
0,0,adj.all
1,1,noun.Tops
2,2,adv.all
3,3,verb.body
4,4,noun.act


### ConceptNet - Edges

In [22]:
relationships = pd.read_csv(root+'WN-edges.csv',index_col=[0])
relationships.head(5)

,:END_ID,:TYPE
:START_ID,,
00001740-a,05207437-n,attribute
00001740-a,05624029-n,attribute
00001740-n,00001930-n,hyponym
00001740-n,00002137-n,hyponym
00001740-n,04431553-n,hyponym


### ConceptNet - Part of Speech

In [23]:
relationships = pd.read_csv(root+'encoded/pos_wn.csv',index_col=[0])
relationships.head()

,type,id
0,adjective,0
1,noun,3
2,adverb,2
3,verb,4
4,adjective_satellite,1


## Define and Publish the Schema

In [24]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
results = conn.gsql('''
  USE GLOBAL
  CREATE VERTEX word(PRIMARY_ID id STRING, uri STRING, name STRING, definition STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX subject(PRIMARY_ID id STRING, type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX pos(PRIMARY_ID id STRING, type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE DIRECTED EDGE has_subject(FROM word, TO subject) WITH REVERSE_EDGE="reverse_has_subject"
  CREATE DIRECTED EDGE part_of_speech(FROM word, TO pos) WITH REVERSE_EDGE="reverse_part_of_speech"
  CREATE DIRECTED EDGE attribute(FROM word, TO word) WITH REVERSE_EDGE="reverse_attribute"
  CREATE DIRECTED EDGE hyponym(FROM word, TO word) WITH REVERSE_EDGE="reverse_hyponym"
  CREATE DIRECTED EDGE entails(FROM word, TO word) WITH REVERSE_EDGE="reverse_entails"
  CREATE DIRECTED EDGE similar(FROM word, TO word) WITH REVERSE_EDGE="reverse_similar"
  CREATE DIRECTED EDGE hypernym(FROM word, TO word) WITH REVERSE_EDGE="reverse_hypernym"
  CREATE DIRECTED EDGE domain_topic(FROM word, TO word) WITH REVERSE_EDGE="reverse_domain_topic"
  CREATE DIRECTED EDGE mero_part(FROM word, TO word) WITH REVERSE_EDGE="reverse_mero_part"
  CREATE DIRECTED EDGE exemplifies(FROM word, TO word) WITH REVERSE_EDGE="reverse_exemplifies"
  CREATE DIRECTED EDGE has_domain_topic(FROM word, TO word) WITH REVERSE_EDGE="reverse_has_domain_topic"
  CREATE DIRECTED EDGE also(FROM word, TO word) WITH REVERSE_EDGE="reverse_also"
  CREATE DIRECTED EDGE mero_substance(FROM word, TO word) WITH REVERSE_EDGE="reverse_mero_substance"
  CREATE DIRECTED EDGE domain_region(FROM word, TO word) WITH REVERSE_EDGE="reverse_domain_region"
  CREATE DIRECTED EDGE holo_part(FROM word, TO word) WITH REVERSE_EDGE="reverse_holo_part"
  CREATE DIRECTED EDGE holo_member(FROM word, TO word) WITH REVERSE_EDGE="reverse_holo_member"
  CREATE DIRECTED EDGE causes(FROM word, TO word) WITH REVERSE_EDGE="reverse_causes"
  CREATE DIRECTED EDGE instance_hyponym(FROM word, TO word) WITH REVERSE_EDGE="reverse_instance_hyponym"
  CREATE DIRECTED EDGE instance_hypernym(FROM word, TO word) WITH REVERSE_EDGE="reverse_instance_hypernym"
  CREATE DIRECTED EDGE mero_member(FROM word, TO word) WITH REVERSE_EDGE="reverse_mero_member"
  CREATE DIRECTED EDGE is_exemplified_by(FROM word, TO word) WITH REVERSE_EDGE="reverse_is_exemplified_by"
  CREATE DIRECTED EDGE holo_substance(FROM word, TO word) WITH REVERSE_EDGE="reverse_holo_substance"
  CREATE DIRECTED EDGE has_domain_region(FROM word, TO word) WITH REVERSE_EDGE="reverse_has_domain_region"
  CREATE DIRECTED EDGE antonym(FROM word, TO word) WITH REVERSE_EDGE="reverse_antonym"

''')
print(results)

Successfully created vertex types: [word].
Successfully created vertex types: [subject].
Successfully created vertex types: [pos].
Successfully created edge types: [has_subject].
Successfully created reverse edge types: [reverse_has_subject].
Successfully created edge types: [part_of_speech].
Successfully created reverse edge types: [reverse_part_of_speech].
Successfully created edge types: [attribute].
Successfully created reverse edge types: [reverse_attribute].
Successfully created edge types: [hyponym].
Successfully created reverse edge types: [reverse_hyponym].
Successfully created edge types: [entails].
Successfully created reverse edge types: [reverse_entails].
Successfully created edge types: [similar].
Successfully created reverse edge types: [reverse_similar].
Successfully created edge types: [hypernym].
Successfully created reverse edge types: [reverse_hypernym].
Successfully created edge types: [domain_topic].
Successfully created reverse edge types: [reverse_domain_topic].

# WordNET

## Create WordNet Graph

In [25]:
results = conn.gsql('CREATE GRAPH WordNet(word, subject, pos, has_subject, part_of_speech, attribute, hyponym, entails, similar, hypernym, domain_topic, mero_part, exemplifies, has_domain_topic, also, mero_substance, domain_region, holo_part, holo_member, causes, instance_hyponym, instance_hypernym, mero_member, is_exemplified_by, holo_substance, has_domain_region, antonym)')
print(results)

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 30.233 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.079 seconds
The graph WordNet is created.


In [26]:
conn.graphname="WordNet"
secret = conn.createSecret()
print(secret)
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
# authToken = 'rc7reopbis1667ksgcppq5v5fb99p6s1'
conn = tg.TigerGraphConnection(host=hostName, graphname="WordNet", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

d8hflvtf0poir74r0es1mblld71pjdgk
r7gtbn7im6onommefobe34ne7omckoag


## Create Loading Jobs

### Word

Let's take a look at what one of our files looks like so we can write a loading job.

In [27]:
# !head -n 2 '/content/drive/MyDrive/LanguageModel/Data_Processing/csv_imports/WN-nodes.csv'

Here it's important to note that the `$0`, `$1` values line up with the columns of your data.
In this example:
- `$0` is the `uri` column,
- `$1` is `id`,
- `$2` is `word`
- and so on

In [28]:
results = conn.gsql('''
  USE GRAPH WordNet
  BEGIN
  CREATE LOADING JOB load_job_WN_edges FOR GRAPH WordNet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE mero_part VALUES($0, $1) WHERE $2 == "mero_part" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE domain_topic VALUES($0, $1) WHERE $2 == "domain_topic" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE hypernym VALUES($0, $1) WHERE $2 == "hypernym" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE similar VALUES($0, $1) WHERE $2 == "similar" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE entails VALUES($0, $1) WHERE $2 == "entails" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE hyponym VALUES($0, $1) WHERE $2 == "hyponym" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE attribute VALUES($0, $1) WHERE $2 == "attribute" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE antonym VALUES($0, $1) WHERE $2 == "antonym" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_domain_region VALUES($0, $1) WHERE $2 == "has_domain_region" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE holo_substance VALUES($0, $1) WHERE $2 == "holo_substance" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE is_exemplified_by VALUES($0, $1) WHERE $2 == "is_exemplified_by" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE mero_member VALUES($0, $1) WHERE $2 == "mero_member" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE instance_hypernym VALUES($0, $1) WHERE $2 == "instance_hypernym" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE instance_hyponym VALUES($0, $1) WHERE $2 == "instance_hyponym" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE causes VALUES($0, $1) WHERE $2 == "causes" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE holo_member VALUES($0, $1) WHERE $2 == "holo_member" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE holo_part VALUES($0, $1) WHERE $2 == "holo_part" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE domain_region VALUES($0, $1) WHERE $2 == "domain_region" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE mero_substance VALUES($0, $1) WHERE $2 == "mero_substance" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE exemplifies VALUES($0, $1) WHERE $2 == "exemplifies" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_domain_topic VALUES($0, $1) WHERE $2 == "has_domain_topic" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE also VALUES($0, $1) WHERE $2 == "also" USING SEPARATOR="\t", HEADER="true", EOL="\n", QUOTE="double";

    }

    CREATE LOADING JOB load_job_WN_nodes_encoded FOR GRAPH WordNet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX word VALUES($1, $0, $2, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_subject VALUES($1, $5) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE part_of_speech VALUES($1, $3) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }


    CREATE LOADING JOB load_job_subject_wn FOR GRAPH WordNet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX subject VALUES($2, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }


  CREATE LOADING JOB load_job_pos_wn FOR GRAPH WordNet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX pos VALUES($2, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }


  END
  ''')
print(results)

Using graph 'WordNet'
Successfully created loading jobs: [load_job_WN_edges].
Successfully created loading jobs: [load_job_WN_nodes_encoded].
Successfully created loading jobs: [load_job_subject_wn].
Successfully created loading jobs: [load_job_pos_wn].


## Load Data

### Words

In [29]:
# Load the posts file wiht the 'load_words' job
load_words = root+'WN-edges.csv'
results = conn.uploadFile(load_words, fileTag='MyDataSource', jobName='load_job_WN_edges')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 293652,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "attribute",
          "validObject": 1278,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0,
          "passedCondition": 1278,
          "failedCondition": 292374,
          "failedConditionLines": [
            1,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16,
            17,
            18,
            19,
            20,
            21,
            22,
            23,
            24,
         

In [30]:
# Load the posts file wiht the 'load_words' job
load_words = root+'encoded/WN-nodes-encoded.csv'
results = conn.uploadFile(load_words, fileTag='MyDataSource', jobName='load_job_WN_nodes_encoded')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 117792,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "word",
          "validObject": 117792,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [
        {
          "typeName": "has_subject",
          "validObject": 117792,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        },
        {
          "typeName": "part_of_speech",
          "validObject": 117792,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "in

### Edges

In [31]:
# Load the posts file wiht the 'load_edges' job
load_edges = root+'encoded/subject_wn.csv'
results = conn.uploadFile(load_edges, fileTag='MyDataSource', jobName='load_job_subject_wn')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 46,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "subject",
          "validObject": 46,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [32]:
# Load the posts file wiht the 'load_edges' job
load_edges = root+'encoded/pos_wn.csv'
results = conn.uploadFile(load_edges, fileTag='MyDataSource', jobName='load_job_pos_wn')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 6,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "pos",
          "validObject": 6,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


## Exploring the Graph

### Get Vertex and Edge Schema

In [33]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticies: ['word', 'subject', 'pos']
Edges: ['has_subject', 'part_of_speech', 'attribute', 'hyponym', 'entails', 'similar', 'hypernym', 'domain_topic', 'mero_part', 'exemplifies', 'has_domain_topic', 'also', 'mero_substance', 'domain_region', 'holo_part', 'holo_member', 'causes', 'instance_hyponym', 'instance_hypernym', 'mero_member', 'is_exemplified_by', 'holo_substance', 'has_domain_region', 'antonym']


In [34]:

print(f"Results for Post vertex")
pprint(conn.getVertexType("word"))

print("-----------------")
print(f"Results for liked edge")
pprint(conn.getEdgeType("type"))


Results for Post vertex
{
  "Config": {
    "TAGGABLE": false,
    "STATS": "OUTDEGREE_BY_EDGETYPE",
    "PRIMARY_ID_AS_ATTRIBUTE": true
  },
  "Attributes": [
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "uri",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    },
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "name",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    },
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "definition",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    }
  ],
  "PrimaryId": {
    "AttributeType": {

## Counting Data

In [35]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 111418 word vertices in the graph
There are 1 subject vertices in the graph
There are 0 pos vertices in the graph
--------------
Edge Counts
There are 3782 has_subject edges in the graph
There are 3782 part_of_speech edges in the graph
There are 1278 attribute edges in the graph
There are 89172 hyponym edges in the graph
There are 408 entails edges in the graph
There are 23176 similar edges in the graph
There are 89172 hypernym edges in the graph
There are 6676 domain_topic edges in the graph
There are 9111 mero_part edges in the graph
There are 970 exemplifies edges in the graph
There are 6676 has_domain_topic edges in the graph
There are 2697 also edges in the graph
There are 797 mero_substance edges in the graph
There are 1341 domain_region edges in the graph
There are 9111 holo_part edges in the graph
There are 12288 holo_member edges in the graph
There are 221 causes edges in the graph
There are 8589 instance_hyponym edges in the graph
There are 8589 instan

## Extracting Data

### Vertex/Edge Set Format

#### Getting a Vertex

In [36]:
results = conn.getVerticesById("word", "15314760-n")
pprint(results)

[
  {
    "v_id": "15314760-n",
    "v_type": "word",
    "attributes": {
      "id": "15314760-n",
      "uri": "http://wordnet-rdf.princeton.edu/id/15314760-n",
      "name": "lead_time",
      "definition": "the time interval between the initiation and the completion of a production process; &quot;the lead times for many publications can vary tremendously&quot;; &quot;planning is an area where lead time can be reduced&quot;@en"
    }
  }
]


#### Or Multiple Vertices

In [37]:
tdf1 = conn.getVerticesById("word", ["15137796-n","15192825-n"])
pprint(tdf1)

[
  {
    "v_id": "15137796-n",
    "v_type": "word",
    "attributes": {
      "id": "15137796-n",
      "uri": "http://wordnet-rdf.princeton.edu/id/15137796-n",
      "name": "period",
      "definition": "an amount of time; &quot;a time period of 30 years&quot;; &quot;hastened the period of time of his recovery&quot;; &quot;Picasso's blue period&quot;@en"
    }
  },
  {
    "v_id": "15192825-n",
    "v_type": "word",
    "attributes": {
      "id": "15192825-n",
      "uri": "http://wordnet-rdf.princeton.edu/id/15192825-n",
      "name": "eve",
      "definition": "the period immediately before something; &quot;on the eve of the French Revolution&quot;@en"
    }
  }
]


#### Count Edges Connected to a Vertex

In [38]:
results = conn.getEdgeCountFrom("word", "15192825-n")
pprint(results)

{
  "has_subject": 1,
  "reverse_has_subject": 0,
  "part_of_speech": 1,
  "reverse_part_of_speech": 0,
  "attribute": 0,
  "reverse_attribute": 0,
  "hyponym": 0,
  "reverse_hyponym": 1,
  "entails": 0,
  "reverse_entails": 0,
  "similar": 0,
  "reverse_similar": 0,
  "hypernym": 1,
  "reverse_hypernym": 0,
  "domain_topic": 0,
  "reverse_domain_topic": 0,
  "mero_part": 0,
  "reverse_mero_part": 0,
  "exemplifies": 0,
  "reverse_exemplifies": 0,
  "has_domain_topic": 0,
  "reverse_has_domain_topic": 0,
  "also": 0,
  "reverse_also": 0,
  "mero_substance": 0,
  "reverse_mero_substance": 0,
  "domain_region": 0,
  "reverse_domain_region": 0,
  "holo_part": 0,
  "reverse_holo_part": 0,
  "holo_member": 0,
  "reverse_holo_member": 0,
  "causes": 0,
  "reverse_causes": 0,
  "instance_hyponym": 0,
  "reverse_instance_hyponym": 0,
  "instance_hypernym": 0,
  "reverse_instance_hypernym": 0,
  "mero_member": 0,
  "reverse_mero_member": 0,
  "is_exemplified_by": 0,
  "reverse_is_exemplified_by

#### Show all Edges Connected to a Vertex

In [39]:
results = conn.getEdges("word", "15192825-n")
pprint(results)

[
  {
    "e_type": "has_subject",
    "directed": true,
    "from_id": "15192825-n",
    "from_type": "word",
    "to_id": "29",
    "to_type": "subject",
    "attributes": {}
  },
  {
    "e_type": "part_of_speech",
    "directed": true,
    "from_id": "15192825-n",
    "from_type": "word",
    "to_id": "3",
    "to_type": "pos",
    "attributes": {}
  },
  {
    "e_type": "reverse_hyponym",
    "directed": true,
    "from_id": "15192825-n",
    "from_type": "word",
    "to_id": "15137796-n",
    "to_type": "word",
    "attributes": {}
  },
  {
    "e_type": "hypernym",
    "directed": true,
    "from_id": "15192825-n",
    "from_type": "word",
    "to_id": "15137796-n",
    "to_type": "word",
    "attributes": {}
  }
]


### As Pandas Dataframe
Supports all of the above in native Pandas Dataframe format.

#### All Vertices of one Type

In [40]:
df1 = conn.getVertexDataframe("word")
print(df1)

              v_id          id  \
0       02439471-a  02439471-a   
1       00261207-r  00261207-r   
2       03137593-a  03137593-a   
3       02900272-a  02900272-a   
4       01764521-a  01764521-a   
...            ...         ...   
117787  02861017-a  02861017-a   
117788  00461643-r  00461643-r   
117789  02737745-a  02737745-a   
117790  00133607-r  00133607-r   
117791  00374152-r  00374152-r   

                                                   uri           name  \
0       http://wordnet-rdf.princeton.edu/id/02439471-a     tongueless   
1       http://wordnet-rdf.princeton.edu/id/00261207-r        onstage   
2       http://wordnet-rdf.princeton.edu/id/03137593-a  translational   
3       http://wordnet-rdf.princeton.edu/id/02900272-a       surgical   
4       http://wordnet-rdf.princeton.edu/id/01764521-a  nonreversible   
...                                                ...            ...   
117787  http://wordnet-rdf.princeton.edu/id/02861017-a      epistemic   
117788 

#### One or More Vertex

In [41]:
df2 = conn.getVertexDataframeById("word", ["15192825-n"])
print(df2)

         v_id          id                                             uri  \
0  15192825-n  15192825-n  http://wordnet-rdf.princeton.edu/id/15192825-n   

  name                                         definition  
0  eve  the period immediately before something; &quot...  


#### Convert Vertex/Edge Set to Dataframe
We'll use the results from the 'Or Multiple Vertices' cell. 

In [42]:
df3 = conn.vertexSetToDataFrame(tdf1)
print(df3)

         v_id          id                                             uri  \
0  15137796-n  15137796-n  http://wordnet-rdf.princeton.edu/id/15137796-n   
1  15192825-n  15192825-n  http://wordnet-rdf.princeton.edu/id/15192825-n   

     name                                         definition  
0  period  an amount of time; &quot;a time period of 30 y...  
1     eve  the period immediately before something; &quot...  


#### Get Edges

In [43]:
df4 = conn.getEdgesDataframe("word", "15192825-n", limit=3)
print(df4)

  from_type     from_id  to_type       to_id
0      word  15192825-n  subject          29
1      word  15192825-n      pos           3
2      word  15192825-n     word  15137796-n


## Path Finding
Find paths between vertices.

Supported are:
- shortestPath - one shortest path between vertices
- allPaths - all paths within the specified edge limit

In [44]:
results = conn.shortestPath([("word", "15192825-n")], [("word", "15161294-n")])
pprint(results)

[{'type': 'word', 'id': '15192825-n'}]
[{'type': 'word', 'id': '15161294-n'}]
[
  {
    "vertices": [
      {
        "v_id": "15137796-n",
        "v_type": "word",
        "attributes": {
          "id": "15137796-n",
          "uri": "http://wordnet-rdf.princeton.edu/id/15137796-n",
          "name": "period",
          "definition": "an amount of time; &quot;a time period of 30 years&quot;; &quot;hastened the period of time of his recovery&quot;; &quot;Picasso's blue period&quot;@en"
        }
      },
      {
        "v_id": "15192825-n",
        "v_type": "word",
        "attributes": {
          "id": "15192825-n",
          "uri": "http://wordnet-rdf.princeton.edu/id/15192825-n",
          "name": "eve",
          "definition": "the period immediately before something; &quot;on the eve of the French Revolution&quot;@en"
        }
      },
      {
        "v_id": "15161294-n",
        "v_type": "word",
        "attributes": {
          "id": "15161294-n",
          "uri": "http:

## Clear the Whole Graph
DANGER ZONE

In [45]:
# conn.gsql('''
# USE GLOBAL
# DROP ALL
# ''')